# Fitness Landscape Analysis

Understanding the **fitness landscape** is crucial for selecting the right optimization algorithm and tuning strategies. 
This notebook explores the ruggedness and structure of the TSP instances using two key techniques:
1.  **Random Walk Autocorrelation**: Measures how correlated neighboring solutions are (ruggedness).
2.  **Fitness-Distance Correlation (FDC)**: Measures how well fitness guides the search towards the global optimum.
3.  **Basin of Attraction Estimation**: Estimates the number and relative size of local optima basins.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# VAMOS imports
from vamos.foundation.problem.tsp import TSPProblem

## 1. Load Problem
We use the VAMOS `TSPProblem` class to load a TSPLIB instance (e.g., `kroA100`).

In [ ]:
# Load kroA100 (100 cities)
problem = TSPProblem(dataset="kroa100")
print(f"Loaded {problem.labels[-1]} ({problem.n_var} cities)")

# Helper to evaluate a single solution's tour length (Objective 0)
def eval_fitness(perm):
    # wrapper for vectorized evaluate
    X = np.array([perm])
    out = {"F": np.zeros((1, 2))}
    problem.evaluate(X, out)
    return out["F"][0, 0] # Return total distance

## 2. Random Walk Analysis (Ruggedness)
We perform a random walk by iteratively applying a random 2-opt Swap (inverting a subsegment) or simpler Swap mutation.
Since TSP neighborhoods are often defined by 2-opt (inversion) or Swap (exchange), we'll use **Swap** for simplicity in the random walk.

The **Autocorrelation Function (ACF)** $\rho(k)$ measures the correlation between fitnesses $k$ steps apart.
- **High $\lambda$ (Correlation Length)** implies a smooth landscape (easy).
- **Low $\lambda$** implies a rugged/random landscape (hard).

In [ ]:
def random_walk(problem, steps=1000):
    n = problem.n_var
    current_perm = np.random.permutation(n)
    fitness_history = []
    
    current_fitness = eval_fitness(current_perm)
    fitness_history.append(current_fitness)
    
    for _ in range(steps):
        # Random Swap Mutation
        idx = np.random.choice(n, 2, replace=False)
        i, j = idx[0], idx[1]
        
        # Create neighbor
        neighbor = current_perm.copy()
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        
        # Evaluate (In a pure random walk, we always accept)
        # Note: Ideally we update efficiently, but full eval is fast enough for 100 cities
        f_new = eval_fitness(neighbor)
        
        current_perm = neighbor
        current_fitness = f_new
        fitness_history.append(current_fitness)
        
    return np.array(fitness_history)

steps = 5000
walk_data = random_walk(problem, steps=steps)

plt.figure(figsize=(10, 4))
plt.plot(walk_data, lw=0.5)
plt.title("Random Walk Fitness Trace")
plt.ylabel("Tour Length")
plt.xlabel("Step")
plt.show()

In [ ]:
def autocorrelation(series, k):
    mean = np.mean(series)
    var = np.var(series)
    n = len(series)
    
    numerator = np.sum((series[:n-k] - mean) * (series[k:] - mean))
    denominator = var * (n - k)
    
    return numerator / denominator if denominator != 0 else 0

lags = np.arange(0, 100, 1)
acf_vals = [autocorrelation(walk_data, l) for l in lags]

plt.figure(figsize=(8, 5))
plt.plot(lags, acf_vals, marker='o', markersize=3)
plt.axhline(0, color='k', linestyle='--', linewidth=0.5)
plt.title("Autocorrelation Function (ACF)")
plt.xlabel("Lag k")
plt.ylabel("Autocorrelation $\rho(k)$")
plt.grid(True, alpha=0.3)
plt.show()

# Correlation Length: roughly where ACF drops to 1/e (~0.37)
correlation_len = np.argmax(np.array(acf_vals) < 0.37)
print(f"Correlation Length ($1/e$): ~{correlation_len} steps")

## 3. Fitness-Distance Correlation (FDC)
Does fitness correlate with distance to the global optimum?

Since we don't have the proven global optimum handy, we will approximate it by finding the **best solution from a large random sample** (or a quick local search run).

In [ ]:
def edge_distance(p1, p2):
    # Count common edges
    # Edges are defined by pairs (u, v)
    def get_edges(p):
        n = len(p)
        edges = set()
        for i in range(n):
            u, v = p[i], p[(i+1)%n]
            if u > v: u, v = v, u
            edges.add((u, v))
        return edges
    
    e1 = get_edges(p1)
    e2 = get_edges(p2)
    
    # Distance = Total Edges - Common Edges
    # Normalized to [0, n]
    return len(e1) - len(e1.intersection(e2))

# 1. Generate Samples
pop_size = 500
X_pop = np.array([np.random.permutation(problem.n_var) for _ in range(pop_size)])

# 2. Evaluate Loop
fitnesses = []
out = {"F": np.zeros((1, 2))}
for i in range(pop_size):
    problem.evaluate(X_pop[i:i+1], out)
    fitnesses.append(out["F"][0, 0])
fitnesses = np.array(fitnesses)

# 3. Identify Target (Best found)
best_idx = np.argmin(fitnesses)
best_sol = X_pop[best_idx]
best_fit = fitnesses[best_idx]

print(f"Best found in random sample: {best_fit:.2f}")

# 4. Compute Distances to Best
distances = [edge_distance(p, best_sol) for p in X_pop]

# 5. Plot FDC
plt.figure(figsize=(8, 6))
sns.regplot(x=distances, y=fitnesses, scatter_kws={'alpha':0.5, 's':10}, line_kws={'color':'red'})
plt.title("Fitness-Distance Correlation (FDC)")
plt.xlabel("Edge Distance to Best Solution")
plt.ylabel("Fitness (Length)")
plt.show()

# Calculate Pearson correlation
r_fdc = np.corrcoef(distances, fitnesses)[0, 1]
print(f"FDC Coefficient r = {r_fdc:.4f}")
if r_fdc > 0.15:
    print("Positive correlation: The problem might be deceptive (fitness leads away from optima) or unstructured relative to this reference.")
elif r_fdc < -0.15:
    print("Negative correlation: Good sign! Better fitness implies closer to optimum.")
else:
    print("Low correlation: The landscape is difficult/neutral.")

## 4. Basin of Attraction Estimation
We estimate number of basins by running multiple simple hill-climbers (Randomized 2-opt) from random starts.

In [ ]:
def simple_hill_climber(problem, max_evals=1000):
    """Randomized 2-opt hill climber."""
    n = problem.n_var
    # Start random
    current_perm = np.random.permutation(n)
    current_fit = eval_fitness(current_perm)
    
    evals = 1
    improved = True
    
    while improved and evals < max_evals:
        improved = False
        # Try stochastic neighbors
        for _ in range(50): 
            idx = np.random.choice(n, 2, replace=False)
            i, j = idx[0], idx[1]
            if i > j: i, j = j, i
            
            # 2-opt inversion
            neighbor = current_perm.copy()
            neighbor[i:j+1] = neighbor[i:j+1][::-1]
            
            # Greedy accept
            f_new = eval_fitness(neighbor)
            evals += 1
            
            if f_new < current_fit:
                current_perm = neighbor
                current_fit = f_new
                improved = True
                break # First improvement
                
    return current_fit, current_perm

n_starts = 50
print(f"Running {n_starts} hill-climbers to sample basins...")
final_fitnesses = []

for _ in tqdm(range(n_starts)):
    f, _ = simple_hill_climber(problem, max_evals=1000)
    final_fitnesses.append(f)

# Group by fitness (proxy for basin identity)
final_fitnesses = np.array(final_fitnesses)
unique_fits, counts = np.unique(final_fitnesses, return_counts=True)

# Sort by basin size
sorted_indices = np.argsort(-counts)
unique_fits = unique_fits[sorted_indices]
counts = counts[sorted_indices]

print(f"Found {len(unique_fits)} distinct local optima (by fitness).")

plt.figure(figsize=(10, 5))
plt.bar(range(len(counts)), counts, color='teal', alpha=0.7)
plt.title("Basin of Attraction Sizes (Estimated by Random Starts)")
plt.xlabel("Basin ID (Sorted by Size)")
plt.ylabel("Frequency of Convergence")
plt.grid(axis='y', alpha=0.3)
plt.show()

if len(counts) > 0 and counts[0] > 1:
    print(f"Largest basin captured {counts[0]/n_starts*100:.1f}% of runs.")
else:
    print("Landscape appears very rugged (multimodal), almost every run found a unique local optimum.")